## Example - Gaussian advection with outflow on multiple quantities

This notebook is the v2.0.0 example of using the common_models advection and pressure grad models with outflow on the Gaussian advection example

In [ ]:
%load_ext autoreload
%autoreload 2
import RMK_support as rmk
from RMK_support.common_models import advection,pressureGrad
from RMK_support import node 
from RMK_support.derivations import NodeDerivation

import numpy as np
import holoviews as hv
import matplotlib.pyplot as plt

### Context initialisation with IO and MPI context setting

In [ ]:
rk = rmk.RMKContext()
rk.IOContext = rmk.IOContext(HDF5Dir="./RMKOutput/RMK_advection_cm/")
rk.mpiContext = rmk.MPIContext(numProcsX=4)

### Grid initialization


In [ ]:
# In normalized length or in meters - defaults to normalized unless isLengthInMeters=True in Grid
xGridWidths = 0.025*np.ones(512)
rk.grid = rmk.Grid(xGridWidths, interpretXGridAsWidths=True)

### Variable container


#### Adding variables


In [ ]:
nInit = 1 + np.exp(-(rk.grid.xGrid-np.mean(rk.grid.xGrid))**2) # A Gaussian perturbation
TInit = np.ones(len(rk.grid.xGrid)) # Constant temperature

n,n_dual = rmk.varAndDual("n",rk.grid,data=nInit) #both variable and its dual
T = rmk.Variable("T",rk.grid,data=TInit,isDerived=True,isCommunicated=False)
G_dual,G = rmk.varAndDual("G",rk.grid,primaryOnDualGrid=True) #the first return value is the primary, so here it is the dual
u_dual,u = rmk.varAndDual("u",rk.grid,primaryOnDualGrid=True,derivation=NodeDerivation("u",node(G_dual)/node(n_dual))) # Here u_dual is a derived using the NodeDerivation

rk.variables.add(n,n_dual,T,G_dual,G,u_dual,u)

### Models 

Here we use common model functions advection and pressureGrad to create advection and pressure gradient models for both the density and momentum equation, which should result in a shock forming that is then drained out of the system.


In [ ]:
massRatio = 1/1836
advectionModelDens = advection(n,G_dual,outflow=(True,True),advectionSpeed=u).rename("adv_dens")
advectionModelFlux = advection(G_dual,u_dual*G_dual,outflow=(True,True),advectionSpeed=u).rename("adv_mom")
pressureGradModel = pressureGrad(G_dual,T*n,massRatio/2).rename("pressure_grad")

rk.models.add(advectionModelDens,advectionModelFlux,pressureGradModel)

### Integrator options

In [ ]:
# the implicit BDE integrator that checks convergence based on the variables n and G_dual
integrator = rmk.BDEIntegrator("BDE",nonlinTol=1e-12,absTol=10.0,convergenceVars=[n,G_dual])
integrationStep = rmk.IntegrationStep("BE",integrator)
integrationStep.add(rk.models) # Add all models in context
rk.integrationScheme = rmk.IntegrationScheme(dt=0.1,steps=integrationStep) #Create a scheme with our single step and a constant integration timestep 0.1
rk.integrationScheme.setFixedNumTimesteps(10000,200) # Run for 10000 steps outputting every 200

#### Generate a LaTeX summary of the ReMKiT1D run 

In [ ]:
rk.generatePDF("Gaussian Advection with Outflow Example")

### Create config file

In [ ]:
rk.writeConfigFile()

### Set global plotting options

In [ ]:
hv.extension('matplotlib')
%matplotlib inline 
plt.rcParams['figure.dpi'] = 150
hv.output(size=150,dpi=150)

### Load data from ReMKiT1D output files

In [ ]:
loadedData = rk.loadSimulation()
dataset = loadedData.dataset

### Check shock formation

In [ ]:
dataName = 'n'

curveDict = {t: hv.Scatter(dataset[dataName][{"t":t}],label='simulation').opts(marker="o",color="r",s=6.0) for t in range(len(dataset.coords["t"].data))}
kdims = [hv.Dimension(('t', 'Time'),unit=dataset.coords["t"].attrs["units"], default=0)]
hv.HoloMap(curveDict,kdims=kdims).opts()